In [18]:
import polars as pl
import json

# import
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI, AzureOpenAI


In [15]:
config = json.load(open('./config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint']  

In [7]:
loader = DirectoryLoader('./reference_labels', glob = '*.txt')
documents  = loader.load()

In [16]:
## saving embeddings
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db = Chroma.from_documents(chunks, OpenAIEmbeddings(openai_api_key=api_key), persist_directory="../Chroma")
db.persist()

## loading them
db = Chroma(persist_directory="../Chroma", embedding_function=OpenAIEmbeddings(openai_api_key=api_key))

In [21]:
client = OpenAI(api_key=api_key)
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "you are a fact finder and you answer in yes, no or unsure"},
        {
            "role": "user",
            "content": """
            Is there any evidence for the association between the use of Lisinopril and Acute Kidney Injury?"""
        }
    ],
    model='gpt-4-1106-preview',
    temperature=0,
)
term = chat_completion.choices[0].message.content


In [22]:
term

'Lisinopril is an angiotensin-converting enzyme (ACE) inhibitor commonly prescribed for the treatment of high blood pressure, heart failure, and other conditions. While ACE inhibitors like lisinopril are generally safe and effective, they can affect kidney function and, in certain situations, may lead to acute kidney injury (AKI).\n\nThe association between ACE inhibitors, including lisinopril, and acute kidney injury is well-documented in medical literature. The risk of AKI is particularly increased in certain populations or under specific conditions, such as:\n\n1. Patients with pre-existing chronic kidney disease.\n2. Individuals with severe congestive heart failure.\n3. Patients with dehydration or volume depletion.\n4. Those undergoing major surgery or during episodes of severe hypotension.\n5. Patients with bilateral renal artery stenosis or stenosis in the artery to a single functioning kidney.\n\nThe mechanism by which lisinopril and other ACE inhibitors can cause AKI is relate